In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

### Mahalanobis distace and Amplify Methods

In [2]:
import numpy as np
from numpy.linalg import inv
import cv2
import tqdm
    
def mahal(img, select = None, mean_pix = None):
    """ Improved Mahalanobis distance algorithm originally written 
        by Brandon Doyle.  This is written with care taken to NOT 
        use Python loops.  The key breakthrough here is numpy.einsum().
        The original attempts to make this concise and fast resulted in really
        weird explosions in RAM usage by NumPy.  The only way to make it settle
        was to loop through each pixel value and apply transformations one at a
        time.
        
        numpy.einsum is basically black magic until you understand it but once
        you do you can make very efficient 1-step operations out of previously
        slow multi-step ones.
        
        Args:
        img:     Input image to compute mahalanobis distance on.
        select:  Number of pixels to randomly select when computing the
                 covariance matrix OR a specified list of indices in the 
                 image.  Specifying the indices outright would be faster
                 if you have a bunch of images that are the same size.  
                 The indices could be reused each time rather than recalculating
                 every time you call the function.
                 
                 If select is 'None', then the every pixel in the image is 
                 included in the sample.
        """
    # Flatten image to just one long array of RGB-valued pixels
    arr = np.reshape(img, (img.shape[0] * img.shape[1], 3))

    # no sampling.  use the entire image
    if select is None:
        select = arr
    else:
        # if 'select' is a number, generate an array of size 'select' containing
        # random pixels in 'arr'.
        # otherwise it should be a list of indices of pixels to choose.
        select = arr[np.random.randint(0, arr.shape[0], select), :] if isinstance(select,int) else arr[select]
            
    # calculate the covariance matrix inverse using the sampled array
    invcovar = inv(np.cov(np.transpose(select)))

    if mean_pix is None:
        # no provided mean RGB vector. Assume we are using the images own 
        # mean RGB value
        meandiff = arr - np.mean(select, axis = 0)
    else:
        meandiff = arr - mean_pix
    
    # calculate the difference between every pixel in 'arr' and the mean RGB vector.
    # if provided, use the given mean RGB vector, otherwise calculate the mean RGB 
    # value of 'select'
    meandiff = arr - (mean_pix if mean_pix is not None else np.mean(select, axis = 0))

    '''
        Formula:
            pp = particular pixel
            mp = mean pixel value
            MD = sqrt( transpose(pp - mp) * (covariance_mat^-1) * (pp - mp) )
        
        You'll notice that I've reversed which side gets transposed.  It's just because the
        pixels are stored as rows at this point (above assumes column vectors) and I've set up
        numpy.einsum() to handle the multiplication properly.
    '''
    
    # calculate the first multiplication. 
    output = np.dot(meandiff, invcovar)

    
    # do literally everything else all in this step, then reshape back to image dimensions and return
    return np.sqrt(np.einsum('ij,ij->i', output, meandiff)).reshape(img.shape[:-1])

def amplify(image, mask, cutoff = 0):
    """ Applies 'mask' to 'image' while cutting off values below a percent
        brightness specified in cuttoff.
        
        Args:
        image:   RGB/BGR image to amplify/apply mask to.
        mask:    Grayscale/monocolor (one number per pixel) "image" to act as a mask.
        cuttoff: Percent brightness below which pixels should be cut off.  This is 
                 referring to the brightness of pixels in 'mask' and the 100% setpoint 
                 is the brighted pixel in 'mask'.
        
        Returns the 'amplified' image."""
    
    # cut off values below specified threshold
    mask_modf = np.greater(mask*100.0/np.amax(mask), cutoff) * mask
    
    # apply mask
    output = np.einsum('ij,ijk->ijk',mask_modf, image)/np.amax(mask_modf)
    
    # scale to 255 (2^8 - 1 = unsigned 8-bit max)
    output *= (255.0/np.amax(image))
    
    return np.uint8(output)

### Image Segmentation

In [3]:
import matplotlib.pyplot as plt
from time import time
import os
from ipywidgets import interact, interactive, interact_manual
    
images = os.listdir('images')

def mahalanobis_segmentation(image):
    pixelcounts = []
    times = []

    # read image
    img = cv2.imread('images/' + image)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # stuff for plots
    pixelcounts.append(img.shape[0] * img.shape[1])
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2)

    # do mahalanobis distance calculation
    t = time()
    filt = mahal(img, None, mean_pix=[202,178,114])
    times.append(time() - t)
        
    # put images in the 'plot buffer' if you will
    ax1.imshow(img, interpolation = 'nearest')
    ax2.imshow(filt, interpolation = 'nearest', cmap = 'Greys_r')
    ax3.imshow(amplify(img, filt, cutoff = 255), interpolation = 'nearest' )
    ax4.imshow(amplify(img, filt, cutoff = 25), interpolation = 'nearest')
        
    plt.show()   


interactive(mahalanobis_segmentation, image = images)

interactive(children=(Dropdown(description='image', options=('CBOTA14STP0001_3.jpg', 'CLC0GAZLJNS1_C.jpg', 'CL…